# 🎯 DeepEval: Evaluating LangChain Agents with LLM-as-Judge

**Tech Stack:** DeepEval · LangChain · Python 3.11+

---

## 📚 What You'll Learn

This notebook demonstrates how to evaluate AI agents using **DeepEval's LLM-as-Judge** approach:

| Section | Topic |
|---------|-------|
| 🔧 Setup | Environment and API configuration |
| 🛠️ Tools | LangChain tools for loan calculations |
| 🤖 Agent | Building a ReAct agent |
| 📝 Goldens | Test templates covering multiple scenarios |
| 📊 Metrics | **Reference-based**, **Referenceless**, and **Agent Tool** metrics |
| 🔬 Evaluation | Running DeepEval on agent outputs |
| 📈 Analysis | Understanding and interpreting results |

---

## 🎓 Key Concept: DeepEval Metric Categories

### 📋 Reference-Based Metrics
> **Require ground truth** (`expected_output` or `retrieval_context`)

| Metric | Required Parameter | What It Measures |
|--------|-------------------|------------------|
| `Faithfulness` | `retrieval_context` | Is output grounded in provided context? |
| `Hallucination` | `context` | Does output contain fabricated info? |

### 🆓 Referenceless Metrics  
> **No ground truth needed** - evaluate output quality directly

| Metric | Required Parameter | What It Measures |
|--------|-------------------|------------------|
| `AnswerRelevancy` | `input` only | Is response relevant to the question? |

### 🔧 Agent Tool Metrics
> **Evaluate tool calling behavior** - specific to agentic systems

| Metric | Required Parameter | What It Measures |
|--------|-------------------|------------------|
| `ToolCorrectness` | `tools_called`, `expected_tools` | Did agent call the right tools? |
| `ArgumentCorrectness` | `tools_called` (with input) | Did agent parse correct parameters? |

---

## 🏦 Demo: Loan Advisor Agent

We'll evaluate a **multi-type loan advisor** with 6 tools:

| Tool | Loan Type | Description |
|------|-----------|-------------|
| `calculate_personal_loan` | 💳 Personal | Unsecured loan payment calculation |
| `calculate_mortgage` | 🏠 Mortgage | Home loan with LTV & PMI analysis |
| `calculate_auto_loan` | 🚗 Auto | Car loan with trade-in support |
| `check_loan_eligibility` | ✅ All | Credit & income eligibility check |
| `check_affordability` | 💰 All | DTI-based affordability analysis |
| `compare_loan_options` | 📊 All | Compare different loan terms |

---

## 🔧 Part 1: Environment Setup

In [ ]:
# 📦 Install dependencies (uncomment for Kaggle/Colab)
# !pip install -q deepeval langchain langchain-openai langgraph pandas python-dotenv

In [ ]:
import os
import json
from typing import Dict, Any

# =============================================================================
# 🔑 API Key Setup
# =============================================================================
# For Kaggle: Add your key to "Add-ons" → "Secrets" → "OPENAI_API_KEY"
# For local/Colab: You'll be prompted to enter your key
# =============================================================================

if not os.getenv("OPENAI_API_KEY"):
    # Try Kaggle Secrets first (for auto-run when saving version)
    try:
        from kaggle_secrets import UserSecretsClient
        os.environ["OPENAI_API_KEY"] = UserSecretsClient().get_secret("OPENAI_API_KEY")
        print("✅ API key loaded from Kaggle Secrets")
    except:
        # Fall back to manual input
        from getpass import getpass
        os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API Key: ")
        print("✅ API key configured")
else:
    print("✅ API key already set")

## 🛠️ Part 2: LangChain Tools Definition

This notebook uses **6 LangChain tools** for loan calculations:

| Tool | Type | Description |
|------|------|-------------|
| `calculate_personal_loan` | 💳 Personal | Unsecured loan payment calculation |
| `calculate_mortgage` | 🏠 Mortgage | Home loan with down payment & LTV ratio |
| `calculate_auto_loan` | 🚗 Auto | Car loan with trade-in support |
| `check_loan_eligibility` | ✅ Check | Credit & income eligibility verification |
| `check_affordability` | 💰 Check | DTI-based affordability analysis |
| `compare_loan_options` | 📊 Compare | Compare different loan terms side-by-side |

> ⬇️ **Next cell:** Run once to create `langchain_tools.py`. You can **collapse it** after running (click the blue bar on the left in Kaggle/Jupyter).

In [ ]:
# 📦 Create langchain_tools.py (run once, then collapse this cell)
# 👆 Click the blue bar on the left to collapse/expand in Kaggle/Jupyter

TOOLS_CODE = '''
import json
from langchain_core.tools import tool

def _calculate_monthly_payment(principal: float, annual_rate: float, months: int) -> float:
    if annual_rate == 0: return principal / months
    monthly_rate = annual_rate / 12
    return round(principal * (monthly_rate * (1 + monthly_rate) ** months) / ((1 + monthly_rate) ** months - 1), 2)

def _calculate_totals(principal: float, monthly_payment: float, months: int) -> dict:
    total_payment = monthly_payment * months
    total_interest = total_payment - principal
    return {"total_payment": round(total_payment, 2), "total_interest": round(total_interest, 2), "interest_percentage": round(total_interest / principal * 100, 2)}

@tool
def calculate_personal_loan(loan_amount: float, annual_interest_rate: float, loan_term_months: int) -> str:
    """Calculate monthly payment for a personal loan. Args: loan_amount (dollars), annual_interest_rate (decimal, e.g. 0.10), loan_term_months."""
    monthly = _calculate_monthly_payment(loan_amount, annual_interest_rate, loan_term_months)
    return json.dumps({"loan_type": "personal", "loan_amount": loan_amount, "annual_interest_rate": annual_interest_rate, "loan_term_months": loan_term_months, "monthly_payment": monthly, **_calculate_totals(loan_amount, monthly, loan_term_months)})

@tool
def calculate_mortgage(home_price: float, down_payment_percent: float, annual_interest_rate: float, loan_term_years: int) -> str:
    """Calculate monthly payment for a mortgage. Args: home_price, down_payment_percent (decimal), annual_interest_rate (decimal), loan_term_years."""
    down_payment = home_price * down_payment_percent
    loan_amount = home_price - down_payment
    loan_term_months = loan_term_years * 12
    ltv_ratio = loan_amount / home_price
    monthly = _calculate_monthly_payment(loan_amount, annual_interest_rate, loan_term_months)
    return json.dumps({"loan_type": "mortgage", "home_price": home_price, "down_payment": round(down_payment, 2), "loan_amount": round(loan_amount, 2), "ltv_ratio": round(ltv_ratio, 3), "annual_interest_rate": annual_interest_rate, "loan_term_years": loan_term_years, "monthly_payment": monthly, **_calculate_totals(loan_amount, monthly, loan_term_months), "ltv_warning": "LTV exceeds 80%, PMI may be required" if ltv_ratio > 0.80 else None})

@tool
def calculate_auto_loan(vehicle_price: float, down_payment: float, annual_interest_rate: float, loan_term_months: int, trade_in_value: float = 0) -> str:
    """Calculate monthly payment for an auto loan. Args: vehicle_price, down_payment (dollars), annual_interest_rate (decimal), loan_term_months, trade_in_value (optional)."""
    effective_price = vehicle_price - trade_in_value
    loan_amount = effective_price - down_payment
    monthly = _calculate_monthly_payment(loan_amount, annual_interest_rate, loan_term_months)
    return json.dumps({"loan_type": "auto", "vehicle_price": vehicle_price, "trade_in_value": trade_in_value, "effective_price": round(effective_price, 2), "down_payment": down_payment, "loan_amount": round(loan_amount, 2), "monthly_payment": monthly, **_calculate_totals(loan_amount, monthly, loan_term_months)})

@tool
def check_loan_eligibility(age: int, monthly_income: float, credit_score: int, employment_status: str, requested_loan_amount: float, loan_type: str = "personal") -> str:
    """Check loan eligibility. Args: age, monthly_income, credit_score (300-850), employment_status (full_time/part_time/self_employed/unemployed), requested_loan_amount, loan_type."""
    reasons, is_eligible = [], True
    if age < 18 or age > 65: is_eligible, reasons = False, reasons + [f"Age {age} outside 18-65 range"]
    else: reasons.append("Age requirement met")
    min_income = 3000 if loan_type == "personal" else 4000
    if monthly_income < min_income: is_eligible, reasons = False, reasons + [f"Income below ${min_income}"]
    else: reasons.append(f"Income OK (${monthly_income:,.0f}/month)")
    min_scores = {"personal": 600, "mortgage": 620, "auto": 580}
    min_score = min_scores.get(loan_type, 600)
    credit_rating = "Poor" if credit_score < min_score else "Fair" if credit_score < 670 else "Good" if credit_score < 740 else "Excellent"
    if credit_score < min_score: is_eligible, reasons = False, reasons + [f"Credit {credit_score} below {min_score}"]
    else: reasons.append(f"Credit {credit_rating} ({credit_score})")
    if employment_status == "unemployed": is_eligible, reasons = False, reasons + ["Employment required"]
    else: reasons.append(f"Employment OK ({employment_status})")
    max_dti = 0.43 if loan_type == "mortgage" else 0.50
    dti = (requested_loan_amount / 36) / monthly_income
    if dti > max_dti: is_eligible, reasons = False, reasons + [f"DTI {dti:.1%} exceeds {max_dti:.0%}"]
    return json.dumps({"loan_type": loan_type, "is_eligible": is_eligible, "credit_rating": credit_rating, "estimated_dti": round(dti, 3), "reasons": reasons, "max_recommended_loan": round(monthly_income * max_dti * 36, 2)})

@tool
def check_affordability(monthly_income: float, existing_monthly_debt: float, proposed_loan_amount: float, annual_interest_rate: float, loan_term_months: int) -> str:
    """Check loan affordability based on DTI. Args: monthly_income, existing_monthly_debt, proposed_loan_amount, annual_interest_rate, loan_term_months."""
    new_payment = _calculate_monthly_payment(proposed_loan_amount, annual_interest_rate, loan_term_months)
    new_dti = (existing_monthly_debt + new_payment) / monthly_income
    assessment = "Excellent" if new_dti <= 0.30 else "Good" if new_dti <= 0.40 else "Acceptable" if new_dti <= 0.50 else "Not recommended"
    return json.dumps({"is_affordable": new_dti <= 0.50, "monthly_income": monthly_income, "existing_monthly_debt": existing_monthly_debt, "new_monthly_payment": new_payment, "total_monthly_debt": round(existing_monthly_debt + new_payment, 2), "current_dti": round(existing_monthly_debt / monthly_income, 3), "new_dti": round(new_dti, 3), "assessment": assessment})

@tool
def compare_loan_options(loan_amount: float, annual_interest_rate: float, term_options: str = "36,48,60") -> str:
    """Compare loan options across different terms. Args: loan_amount, annual_interest_rate, term_options (comma-separated months)."""
    terms = [int(t.strip()) for t in term_options.split(",")]
    comparisons = []
    for months in terms:
        monthly = _calculate_monthly_payment(loan_amount, annual_interest_rate, months)
        totals = _calculate_totals(loan_amount, monthly, months)
        comparisons.append({"term_months": months, "monthly_payment": monthly, "total_interest": totals["total_interest"]})
    comparisons.sort(key=lambda x: x["term_months"])
    if len(comparisons) > 1:
        for c in comparisons: c["interest_savings_vs_longest"] = round(comparisons[-1]["total_interest"] - c["total_interest"], 2)
    return json.dumps({"loan_amount": loan_amount, "annual_interest_rate": annual_interest_rate, "comparisons": comparisons})

def get_all_tools():
    return [calculate_personal_loan, calculate_mortgage, calculate_auto_loan, check_loan_eligibility, check_affordability, compare_loan_options]

def get_tool_descriptions() -> str:
    return "\\n".join([f"- **{t.name}**: {t.description.split(chr(46))[0]}" for t in get_all_tools()])
'''

# Write to file
with open("langchain_tools.py", "w") as f:
    f.write(TOOLS_CODE.strip())
print("✅ Created langchain_tools.py")

In [ ]:
# 📥 Import the tools we just created
from langchain_tools import get_all_tools, get_tool_descriptions

tools = get_all_tools()
print(f"✅ Loaded {len(tools)} tools:\n")
print(get_tool_descriptions())

# 🔧 Create tool descriptions mapping for ArgumentCorrectnessMetric
TOOL_DESCRIPTIONS = {tool.name: tool.description for tool in tools}
print(f"\n📋 Tool descriptions captured for ArgumentCorrectnessMetric")

## 🤖 Part 3: Create LangChain ReAct Agent

We use a **ReAct (Reason + Act) agent** pattern that:
1. Receives user input
2. Decides which tool(s) to call
3. Executes tools and observes results
4. Generates final response

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 🧠 Create LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 📋 System prompt defines agent behavior
SYSTEM_PROMPT = """You are a Loan Advisor assistant helping users with:
- Personal loans, mortgages (home loans), and auto (car) loans
- Payment calculations, eligibility checks, and affordability analysis

Use the provided tools for accurate calculations. Be clear and helpful.
Always show the key numbers in your response."""

# 🤖 Create ReAct agent
agent = create_react_agent(llm, tools, prompt=SYSTEM_PROMPT)
print("🤖 Agent created with 6 loan advisor tools")

In [ ]:
# 🏃 Agent Runner - extracts outputs and tool context
class AgentRunner:
    """Wrapper that runs agent and extracts outputs for evaluation."""
    
    def __init__(self, agent, tool_descriptions: dict):
        self.agent = agent
        self.tool_descriptions = tool_descriptions  # 🔧 For ArgumentCorrectnessMetric
    
    def run(self, query: str) -> Dict[str, Any]:
        """Run agent and extract actual_output + retrieval_context + tools_called with args."""
        result = self.agent.invoke({"messages": [HumanMessage(content=query)]})
        messages = result["messages"]
        
        # 📤 Extract final AI response
        actual_output = next(
            (m.content for m in reversed(messages) if isinstance(m, AIMessage) and m.content),
            ""
        )
        
        # 🔧 Extract tool calls with full details for both metrics
        tools_called = []      # List of tool names
        tools_with_args = []   # List of dicts with name, description, input (args)
        retrieval_context = []
        
        for msg in messages:
            if isinstance(msg, AIMessage) and msg.tool_calls:
                for tc in msg.tool_calls:
                    if isinstance(tc, dict):
                        tool_name = tc.get("name", "")
                        tool_args = tc.get("args", {})  # 🔧 Capture arguments!
                        
                        tools_called.append(tool_name)
                        tools_with_args.append({
                            "name": tool_name,
                            "description": self.tool_descriptions.get(tool_name, ""),
                            "input": tool_args,  # 🔧 For ArgumentCorrectnessMetric
                        })
            if isinstance(msg, ToolMessage):
                retrieval_context.append(msg.content)
        
        return {
            "actual_output": actual_output,
            "tools_called": tools_called,        # 🔧 For ToolCorrectnessMetric
            "tools_with_args": tools_with_args,  # 🔧 For ArgumentCorrectnessMetric
            "retrieval_context": retrieval_context,
        }

runner = AgentRunner(agent, TOOL_DESCRIPTIONS)
print("✅ AgentRunner ready (captures tool arguments for ArgumentCorrectnessMetric)")

## 📝 Part 4: Define Test Goldens

**Goldens** = test templates that define:
- `input`: User query to test
- `expected_tools`: Which tools should be called
- `expected_keywords`: Key terms that should appear in response

We cover **5 categories** across all loan types:

In [ ]:
GOLDENS = [
    # ==========================================================================
    # 💳 PERSONAL LOAN
    # ==========================================================================
    {
        "id": "personal_loan_basic",
        "category": "💳 Personal",
        "input": "Calculate monthly payment for a $25,000 personal loan at 10% interest for 48 months.",
        "expected_tools": ["calculate_personal_loan"],
        "expected_keywords": ["634", "monthly", "payment"],
    },
    {
        "id": "personal_loan_comparison",
        "category": "💳 Personal",
        "input": "Compare a $20,000 personal loan at 9% interest for 36, 48, and 60 months.",
        "expected_tools": ["compare_loan_options"],
        "expected_keywords": ["36", "48", "60", "interest"],
    },
    
    # ==========================================================================
    # 🏠 MORTGAGE (HOME LOAN)
    # ==========================================================================
    {
        "id": "mortgage_basic",
        "category": "🏠 Mortgage",
        "input": "Calculate mortgage payment for a $500,000 home with 20% down payment at 6.5% for 30 years.",
        "expected_tools": ["calculate_mortgage"],
        "expected_keywords": ["2,528", "monthly", "down payment", "400,000"],
    },
    {
        "id": "mortgage_low_down",
        "category": "🏠 Mortgage",
        "input": "What's the monthly payment for a $400,000 house with only 10% down at 7% for 30 years? Will I need PMI?",
        "expected_tools": ["calculate_mortgage"],
        "expected_keywords": ["LTV", "PMI", "monthly"],
    },
    
    # ==========================================================================
    # 🚗 AUTO LOAN (CAR LOAN)
    # ==========================================================================
    {
        "id": "auto_loan_basic",
        "category": "🚗 Auto",
        "input": "Calculate car loan payment for a $35,000 vehicle with $5,000 down at 5.9% for 60 months.",
        "expected_tools": ["calculate_auto_loan"],
        "expected_keywords": ["581", "monthly", "30,000"],
    },
    {
        "id": "auto_loan_trade_in",
        "category": "🚗 Auto",
        "input": "I want to buy a $40,000 car. I have a trade-in worth $8,000 and can put $2,000 down. What's my payment at 6% for 72 months?",
        "expected_tools": ["calculate_auto_loan"],
        "expected_keywords": ["trade", "monthly", "30,000"],
    },
    
    # ==========================================================================
    # ✅ ELIGIBILITY CHECK
    # ==========================================================================
    {
        "id": "eligibility_good_credit",
        "category": "✅ Eligibility",
        "input": "Check my loan eligibility: age 35, income $8,000/month, credit score 750, full-time employed, requesting $50,000 personal loan.",
        "expected_tools": ["check_loan_eligibility"],
        "expected_keywords": ["eligible", "Excellent", "750"],
    },
    {
        "id": "eligibility_low_credit",
        "category": "✅ Eligibility",
        "input": "Am I eligible for a mortgage? Age 28, income $5,000/month, credit score 580, self-employed, want $300,000.",
        "expected_tools": ["check_loan_eligibility"],
        "expected_keywords": ["not", "credit", "580"],
    },
    
    # ==========================================================================
    # 💰 AFFORDABILITY ANALYSIS
    # ==========================================================================
    {
        "id": "affordability_ok",
        "category": "💰 Affordability",
        "input": "Can I afford a $30,000 car loan at 6% for 60 months? I earn $6,000/month with $500 existing debt.",
        "expected_tools": ["check_affordability"],
        "expected_keywords": ["affordable", "DTI"],
    },
    {
        "id": "affordability_stretched",
        "category": "💰 Affordability",
        "input": "Monthly income $4,000, existing debt $1,500. Can I afford a $25,000 loan at 8% for 48 months?",
        "expected_tools": ["check_affordability"],
        "expected_keywords": ["DTI", "exceed", "not"],
    },
]

# 📊 Summary
print(f"📝 Defined {len(GOLDENS)} test cases:\n")
from collections import Counter
categories = Counter(g["category"] for g in GOLDENS)
for cat, count in categories.items():
    print(f"  {cat}: {count} tests")

## 🏃 Part 5: Run Agent on All Test Cases

Execute the agent for each Golden and collect:
- `actual_output`: Agent's final response
- `retrieval_context`: Tool outputs (used by Faithfulness/Hallucination metrics)

In [ ]:
from deepeval.dataset import EvaluationDataset, Golden

print("🏃 Running agent on all test cases...\n")
print("=" * 60)

goldens_with_output = []

for g in GOLDENS:
    print(f"\n{g['category']} | {g['id']}")
    print(f"  📥 Input: {g['input'][:50]}...")
    
    result = runner.run(g["input"])
    print(f"  🔧 Tools called: {result['tools_called']}")
    
    # Create Golden with agent output
    golden = Golden(
        input=g["input"],
        actual_output=result["actual_output"],
        retrieval_context=result["retrieval_context"],  # 💡 Tool outputs!
        additional_metadata={
            "test_id": g["id"],
            "category": g["category"],
            "expected_tools": g["expected_tools"],
            "actual_tools": result["tools_called"],
            "tools_with_args": result["tools_with_args"],  # 🔧 For ArgumentCorrectness
            "expected_keywords": g["expected_keywords"],
        }
    )
    goldens_with_output.append(golden)

print("\n" + "=" * 60)
dataset = EvaluationDataset(goldens=goldens_with_output)
print(f"\n✅ Created dataset with {len(dataset.goldens)} test cases")
print("   Captured: tool names, arguments, and descriptions for evaluation")

## 📊 Part 6: Configure DeepEval Metrics

We use **5 metrics** across three categories:

| Metric | Category | Uses Context? | What It Measures |
|--------|----------|---------------|------------------|
| `AnswerRelevancy` | 🆓 Referenceless | ❌ No | Is response relevant to the question? |
| `Faithfulness` | 📋 Reference-based | ✅ Yes | Is response grounded in tool outputs? |
| `Hallucination` | 📋 Reference-based | ✅ Yes | Does response make things up? |
| `ToolCorrectness` | 🔧 Agent Tool | Uses `expected_tools` | Did agent call the right tools? |
| `ArgumentCorrectness` | 🔧 Agent Tool | Uses `tools_called.input` | Did agent parse correct parameters? |

> 💡 **Why these metrics?**  
> - `AnswerRelevancy`: Ensures agent addresses the user's question  
> - `Faithfulness`: Ensures agent uses tool outputs correctly  
> - `Hallucination`: Catches fabricated numbers or facts
> - `ToolCorrectness`: Validates agent's tool selection behavior
> - `ArgumentCorrectness`: Validates agent extracts correct parameters from user input

In [ ]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric, 
    FaithfulnessMetric, 
    HallucinationMetric,
    ToolCorrectnessMetric,      # 🔧 Agent tool metric (deterministic)
    ArgumentCorrectnessMetric,  # 🔧 Agent tool metric (LLM-based)
)
from deepeval.test_case import LLMTestCase, ToolCall  # 🔧 ToolCall for agent metrics
from deepeval.tracing.api import MetricData  # 🔑 Native Pydantic model for serialization

# 🧠 Judge model for evaluation
EVAL_MODEL = "gpt-4o-mini"

# 📊 Configure metrics
metrics = [
    # 🆓 REFERENCELESS: Only needs input + actual_output
    AnswerRelevancyMetric(threshold=0.7, model=EVAL_MODEL),
    
    # 📋 REFERENCE-BASED: Need retrieval_context (tool outputs)
    FaithfulnessMetric(threshold=0.7, model=EVAL_MODEL),
    HallucinationMetric(threshold=0.5, model=EVAL_MODEL),
    
    # 🔧 AGENT TOOL: Validates tool calling behavior
    ToolCorrectnessMetric(threshold=0.5),  # Deterministic, no LLM needed
    ArgumentCorrectnessMetric(threshold=0.7, model=EVAL_MODEL),  # LLM evaluates args
]

print("📊 Metrics configured:\n")
print("  🆓 Referenceless (no ground truth needed):")
print(f"     • AnswerRelevancy (threshold: 0.7)")
print("\n  📋 Reference-based (uses retrieval_context):")
print(f"     • Faithfulness (threshold: 0.7)")
print(f"     • Hallucination (threshold: 0.5)")
print("\n  🔧 Agent Tool Metrics:")
print(f"     • ToolCorrectness (threshold: 0.5, deterministic)")
print(f"     • ArgumentCorrectness (threshold: 0.7, LLM-based)")
print("\n💾 Using native MetricData for serialization/deserialization")

In [ ]:
# 🔄 Convert Goldens to LLMTestCases
test_cases = []
for g in dataset.goldens:
    meta = g.additional_metadata
    
    # 🔧 For ToolCorrectnessMetric: simple ToolCall with name only
    expected_tools = [ToolCall(name=name) for name in meta["expected_tools"]]
    
    # 🔧 For ArgumentCorrectnessMetric: full ToolCall with name, description, input
    tools_called = [
        ToolCall(
            name=tc["name"],
            description=tc["description"],
            input=tc["input"],  # 🔧 The extracted parameters!
        )
        for tc in meta["tools_with_args"]
    ]
    
    test_case = LLMTestCase(
        input=g.input,
        actual_output=g.actual_output,
        context=g.retrieval_context,           # 📋 For Hallucination metric
        retrieval_context=g.retrieval_context,  # 📋 For Faithfulness metric
        tools_called=tools_called,              # 🔧 Full ToolCall for ArgumentCorrectness
        expected_tools=expected_tools,          # 🔧 For ToolCorrectness metric
    )
    test_cases.append(test_case)

print(f"✅ Created {len(test_cases)} LLMTestCases")
print("\n💡 LLMTestCase fields used:")
print("   • retrieval_context = tool outputs (for Faithfulness/Hallucination)")
print("   • tools_called = ToolCall(name, description, input) (for ArgumentCorrectness)")
print("   • expected_tools = ToolCall(name) (for ToolCorrectness)")

## 🔬 Part 7: Run DeepEval Evaluation

Now we run the LLM-as-Judge evaluation on all test cases.

> 💾 **Native JSON Export**: Using DeepEval's Pydantic `MetricData.model_dump()` for serialization.
> 
> DeepEval's `MetricData` is a Pydantic BaseModel with fields:
> - `name`, `score`, `success`, `threshold`, `reason`, `evaluation_model`, etc.

In [ ]:
print("🔬 Running DeepEval evaluation...")
print("   This may take a few minutes as the judge LLM evaluates each test case.\n")

# 🚀 Run evaluation
results = evaluate(
    test_cases=test_cases,
    metrics=metrics,
)

print(f"\n✅ Evaluation complete!")

# 💾 Save results using DeepEval's native Pydantic serialization
# MetricData is a Pydantic BaseModel with model_dump() method
RESULTS_FILE = "eval_results.json"

native_results = []
for i, golden in enumerate(dataset.goldens):
    meta = golden.additional_metadata
    test_result = results.test_results[i]
    
    # Build result dict with native MetricData serialization
    result_entry = {
        "test_id": meta["test_id"],
        "category": meta["category"],
        "input": golden.input,
        "success": test_result.success,
        # 🔑 Native Pydantic serialization: MetricData.model_dump()
        "metrics": [
            metric_data.model_dump() for metric_data in test_result.metrics_data
        ] if test_result.metrics_data else []
    }
    native_results.append(result_entry)

# Save to JSON
with open(RESULTS_FILE, "w") as f:
    json.dump(native_results, f, indent=2, default=str)

print(f"💾 Results saved to {RESULTS_FILE} (using MetricData.model_dump())")
print(f"   Contains {len(native_results)} test results with full metric details")

## 📈 Part 8: Analyze Results

Load results using DeepEval's native Pydantic deserialization.

> 💾 **Fully Native Approach**:
> - **Save**: `MetricData.model_dump()` → JSON
> - **Load**: `MetricData.model_validate()` → reconstruct objects
> 
> This ensures type safety and schema validation through Pydantic.

In [ ]:
import pandas as pd

# 📊 Load native JSON results
with open(RESULTS_FILE, "r") as f:
    native_results = json.load(f)

# 🔄 Transform using native MetricData.model_validate()
results_data = []
for entry in native_results:
    row = {
        "test_id": entry["test_id"],
        "category": entry["category"],
    }
    
    # 🔑 Reconstruct native MetricData objects using Pydantic
    for metric_dict in entry["metrics"]:
        metric = MetricData.model_validate(metric_dict)  # Native deserialization!
        
        # Access attributes from native MetricData object
        # Handle different metric name formats for cleaner column names
        name = metric.name.replace("Metric", "").replace(" ", "")
        if name == "ToolCorrectness":
            name = "ToolCorrect"
        elif name == "ArgumentCorrectness":
            name = "ArgCorrect"
        row[name] = round(metric.score, 2) if metric.score is not None else 0.0
        row[f"{name}_pass"] = "✅" if metric.success else "❌"
    
    results_data.append(row)

df = pd.DataFrame(results_data)

print("📊 RESULTS BY TEST CASE (loaded via MetricData.model_validate())\n")
print(df.to_string(index=False))

# 📋 Demonstrate native object reconstruction
print(f"\n" + "=" * 50)
print("📋 Native MetricData Object Example:")
sample_metric = MetricData.model_validate(native_results[0]["metrics"][0])
print(f"   Type: {type(sample_metric).__name__}")
print(f"   Name: {sample_metric.name}")
print(f"   Score: {sample_metric.score}")
print(f"   Success: {sample_metric.success}")
print(f"   Threshold: {sample_metric.threshold}")
print(f"   Reason: {sample_metric.reason[:80]}..." if sample_metric.reason else "   Reason: None")

In [ ]:
# 📊 Summary by category
print("📊 AVERAGE SCORES BY CATEGORY\n")
score_cols = ["AnswerRelevancy", "Faithfulness", "Hallucination", "ToolCorrect", "ArgCorrect"]
summary = df.groupby("category")[score_cols].mean().round(2)
print(summary.to_string())

# 🎨 Visual representation
print("\n" + "=" * 50)
print("📈 CATEGORY PERFORMANCE")
print("=" * 50)
for cat in summary.index:
    scores = summary.loc[cat]
    avg = scores.mean()
    bar = "█" * int(avg * 20) + "░" * (20 - int(avg * 20))
    print(f"\n{cat}")
    print(f"  {bar} {avg:.0%}")

In [ ]:
# 📊 Overall pass rates
print("📊 OVERALL PASS RATES\n")

pass_data = {
    "🆓 AnswerRelevancy": (df["AnswerRelevancy_pass"] == "✅").mean(),
    "📋 Faithfulness": (df["Faithfulness_pass"] == "✅").mean(),
    "📋 Hallucination": (df["Hallucination_pass"] == "✅").mean(),
    "🔧 ToolCorrectness": (df["ToolCorrect_pass"] == "✅").mean(),
    "🔧 ArgCorrectness": (df["ArgCorrect_pass"] == "✅").mean(),
}

for metric, rate in pass_data.items():
    bar = "█" * int(rate * 20) + "░" * (20 - int(rate * 20))
    status = "✅" if rate >= 0.8 else "⚠️" if rate >= 0.6 else "❌"
    print(f"  {metric}: {bar} {rate:.0%} {status}")

# Overall
all_pass = (
    (df["AnswerRelevancy_pass"] == "✅") & 
    (df["Faithfulness_pass"] == "✅") & 
    (df["Hallucination_pass"] == "✅") &
    (df["ToolCorrect_pass"] == "✅") &
    (df["ArgCorrect_pass"] == "✅")
).mean()
print(f"\n  🎯 All metrics pass: {all_pass:.0%}")

## 🔧 Part 9: Understanding Agent Tool Metrics

DeepEval provides two specialized metrics for evaluating agent tool-calling behavior:

### ToolCorrectnessMetric (Deterministic)
- Compares `tools_called` vs `expected_tools`
- **No LLM needed** - purely deterministic comparison
- Score = `Correctly Used Tools / Total Tools Called`

### ArgumentCorrectnessMetric (LLM-based)
- Evaluates if agent extracted **correct parameters** from user input
- Uses LLM to judge if `tools_called.input` matches the task
- Score = `Correctly Generated Input Parameters / Total Tool Calls`

> 💡 Both metrics use DeepEval's native `ToolCall` object with `name`, `description`, and `input` fields.

In [ ]:
# 🔧 Detailed Agent Tool Metrics Analysis
print("🔧 AGENT TOOL METRICS BY TEST CASE\n")

for golden in dataset.goldens:
    meta = golden.additional_metadata
    expected = meta["expected_tools"]
    actual = meta["actual_tools"]
    tools_with_args = meta["tools_with_args"]
    
    tool_match = set(expected).issubset(set(actual))
    status = "✅" if tool_match else "❌"
    
    print(f"{status} {meta['test_id']}")
    print(f"   📋 Expected tools: {expected}")
    print(f"   🔧 Called tools:   {actual}")
    
    # Show extracted parameters for ArgumentCorrectness insight
    for tc in tools_with_args:
        print(f"   📥 {tc['name']} args: {tc['input']}")
    print()

# Summary
print("=" * 50)
tool_accuracy = df["ToolCorrect_pass"].value_counts(normalize=True).get("✅", 0)
arg_accuracy = df["ArgCorrect_pass"].value_counts(normalize=True).get("✅", 0)
print(f"🎯 ToolCorrectness Pass Rate: {tool_accuracy:.0%}")
print(f"🎯 ArgumentCorrectness Pass Rate: {arg_accuracy:.0%}")

## 🎓 Part 10: Key Takeaways

### 🔄 DeepEval Evaluation Flow

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Goldens   │ ──► │   Agent     │ ──► │ LLMTestCase │ ──► │  Metrics    │
│ (templates) │     │ Execution   │     │             │     │             │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
                          │
                          ▼
                   Tool Outputs = retrieval_context
                   Tool Args = tools_called.input
```

---

### 💾 Native Serialization with Pydantic

DeepEval's `MetricData` is a Pydantic BaseModel, enabling native serialization:

```python
# Save: MetricData → dict → JSON
metric_data.model_dump()

# Load: JSON → dict → MetricData
MetricData.model_validate(metric_dict)
```

---

### 📊 Metric Categories Summary

| Category | Metrics | Required Parameters | Use Case |
|----------|---------|---------------------|----------|
| 🆓 **Referenceless** | AnswerRelevancy | `input`, `actual_output` only | No labeled data needed |
| 📋 **Reference-based** | Faithfulness, Hallucination | + `retrieval_context` | Ground truth from tools |
| 🔧 **Agent Tool** | ToolCorrectness | `expected_tools` | Validate tool selection |
| 🔧 **Agent Tool** | ArgumentCorrectness | `tools_called.input` | Validate parameter extraction |

---

### ✅ Best Practices

1. **🔧 Tool outputs → retrieval_context**  
   Use tool results as ground truth for Faithfulness/Hallucination

2. **🔧 ToolCorrectness for agents**  
   Validate that your agent selects the right tools (deterministic)

3. **🔧 ArgumentCorrectness for agents**  
   Validate that your agent extracts correct parameters (LLM-based)

4. **💾 Native Pydantic serialization**  
   Use `model_dump()` / `model_validate()` for results persistence

5. **🏷️ Category-based Goldens**  
   Organize tests by loan type/scenario for better analysis

---

### 📚 Resources

- [DeepEval Documentation](https://docs.confident-ai.com/)
- [DeepEval Metrics Guide](https://deepeval.com/docs/metrics-introduction)
- [Tool Correctness Metric](https://deepeval.com/docs/metrics-tool-correctness)
- [Argument Correctness Metric](https://deepeval.com/docs/metrics-argument-correctness)
- [LangChain](https://python.langchain.com/)

---

**👍 Found this helpful? Please upvote!**